custom layered cnn implementation trial with dummy data first

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization, 
    Dropout, Flatten, Dense, LeakyReLU, ReLU
)
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.base import ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score, classification_report

class SklearnKerasClassifier(KerasClassifier, ClassifierMixin):
    def __init__(self, model=None, **kwargs):
        super().__init__(model=model, **kwargs)
    
    @property
    def _tags(self):
        return self.model._tags if hasattr(self.model, "_tags") else {"binary_only": True}

def create_custom_cnn(
    input_shape=None,
    conv_blocks=((32, (3,3)), (64, (3,3))),
    dense_layers=(128,),
    dropout_rate=0.4,
    activation='relu',
    meta=None
):
    """Conv -> BN -> Activation -> Pool -> Dropout"""
    if input_shape is None:
        if meta is None: raise ValueError("meta veya input_shape parametresi yok")
        input_shape = meta["X_shape_"][1:]

    model = Sequential(name="Custom_CNN")
    model.add(Input(shape=input_shape))

    for filters, kernel_size in conv_blocks:
        model.add(Conv2D(filters, kernel_size, padding='same'))
        model.add(BatchNormalization())
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(ReLU())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout_rate))
    model.add(Flatten())
    
    for units in dense_layers:
        model.add(Dense(units))
        model.add(BatchNormalization())
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(ReLU())
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    return model


print("dummy veri seti ile deneme:")
n_samples = 200
img_height, img_width, channels = 128, 128, 3
X = np.random.rand(n_samples, img_height, img_width, channels).astype(np.float32)
y = np.random.randint(0, 2, n_samples)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
print(f"train shape: {X_train.shape} - test shape: {X_test.shape}")

keras_cnn_estimator = SklearnKerasClassifier(
    model=create_custom_cnn,
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam,
    optimizer__learning_rate=0.001,
    epochs=10,
    batch_size=16,
    verbose=0,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0, restore_best_weights=True)]
)

param_grid = {
    'model__conv_blocks': [
        ((16, (3,3)),),                           
        ((32, (3,3)), (64, (3,3))),           
        ((16, (5,5)), (32, (3,3))),
    ],
    'model__dense_layers': [(64,), (128, 64)],
    'model__dropout_rate': [0.3, 0.5],
    'model__activation': ['relu', 'leaky_relu'],
    'optimizer__learning_rate': [0.001, 0.0005]
}

cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
search_cv = RandomizedSearchCV(
    estimator=keras_cnn_estimator,
    param_distributions=param_grid,
    n_iter=4,
    cv=cv_strategy,
    scoring='f1',
    verbose=2,
    random_state=42
)

try:
    search_cv.fit(X_train, y_train, validation_split=0.2)
    print("\n--- hp opt results: ---")
    print(f"F1 (CV): {search_cv.best_score_:.4f}")
    print("best_params_:")
    print(search_cv.best_params_)
    
    best_model = search_cv.best_estimator_
    y_pred = best_model.predict(X_test)
    
    print("\n--- opt test results: ---")
    print(f"accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"F1: {f1_score(y_test, y_pred):.4f}")
    print("\nclass report:")
    print(classification_report(y_test, y_pred))

except Exception as e:
    print(f"\n{e}")
    import traceback
    traceback.print_exc()

dummy veri seti ile deneme:
train shape: (150, 128, 128, 3) - test shape: (50, 128, 128, 3)
Fitting 3 folds for each of 4 candidates, totalling 12 fits


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__conv_blocks=((16, (3, 3)),), model__dense_layers=(64,), model__dropout_rate=0.5, optimizer__learning_rate=0.0005; total time=   6.6s


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__conv_blocks=((16, (3, 3)),), model__dense_layers=(64,), model__dropout_rate=0.5, optimizer__learning_rate=0.0005; total time=   6.1s


...

C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__conv_blocks=((16, (5, 5)), (32, (3, 3))), model__dense_layers=(64,), model__dropout_rate=0.5, optimizer__learning_rate=0.0005; total time=   4.9s


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



--- hp opt results: ---
F1 (CV): 0.6233
best_params_:
{'optimizer__learning_rate': 0.001, 'model__dropout_rate': 0.3, 'model__dense_layers': (64,), 'model__conv_blocks': ((16, (5, 5)), (32, (3, 3))), 'model__activation': 'leaky_relu'}

--- opt test results: ---
accuracy: 0.5000
F1: 0.6667

class report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.50      1.00      0.67        25

    accuracy                           0.50        50
   macro avg       0.25      0.50      0.33        50
weighted avg       0.25      0.50      0.33        50



C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\